In [1]:
import cvxpy as cp
import numpy as np
import scipy
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import vectorized_sdr as vect

In [2]:
N_omega = 2
n = 0
W_plus = np.eye(N_omega)
W_minus = np.eye(N_omega)
J = 0.25*(W_plus@W_plus.conj().T + W_minus@W_minus.conj().T - 2*np.eye(N_omega))
X = np.vstack([J, W_plus, W_minus])
vect_W_plus = vect.mat_to_vec(W_plus)
vect_W_minus = vect.mat_to_vec(W_minus)
vect_J = vect.mat_to_vec(J)
x = sparse.vstack([vect_J, vect_W_plus, vect_W_minus])
y = sparse.vstack([x, 1])

In [3]:
proj = np.zeros((N_omega, N_omega))
lin_terms = []
cst_terms = []
J_def_quad = []
cst_terms_def = []
lin_terms_W_plus_list = []
lin_terms_W_minus_list = []
lin_terms_J_def = []
for i in range(N_omega):
    for j in range(N_omega):
        projection = proj.copy()
        projection[i, j] = 1
        cst_terms.append(np.trace(projection))
        lin_terms.append(vect.mat_to_vec(projection))
        diags = np.zeros((3*(N_omega**2), 3*(N_omega**2)))
        diags[N_omega**2:2*(N_omega**2), N_omega**2:2*(N_omega**2)] = np.kron(np.eye(N_omega), projection)
        diags[2*(N_omega**2):3*(N_omega**2), 2*(N_omega**2):3*(N_omega**2)] = np.kron(np.eye(N_omega), projection)
        J_def_quad.append(sparse.csc_matrix(diags))
        cst_terms_def.append(-2*np.trace(projection))
for i in range(N_omega**2):
    lin_terms_W_plus = lin_terms[i].copy()
    lin_terms_W_plus_list.append(sparse.vstack([sparse.csc_matrix((N_omega**2, 1)), lin_terms_W_plus, sparse.csc_matrix((N_omega**2, 1))]))
    lin_terms_W_minus_list.append(sparse.vstack([sparse.csc_matrix((N_omega**2, 1)), sparse.csc_matrix((N_omega**2, 1)), lin_terms_W_plus]))
    lin_terms_J_def.append(sparse.vstack([-4*lin_terms_W_plus, sparse.csc_matrix((N_omega**2, 1)), sparse.csc_matrix((N_omega**2, 1))]))

In [4]:
sdr_mat_W_plus = [sparse.bmat([[sparse.csc_matrix((3*(N_omega**2),3*(N_omega**2))), 0.5*lin_terms_W_plus_list[i]],[0.5*lin_terms_W_plus_list[i].conj().T, -cst_terms[i]]]) for i in range(len(lin_terms_W_plus_list))]
sdr_mat_W_minus = [sparse.bmat([[sparse.csc_matrix((3*(N_omega**2),3*(N_omega**2))), 0.5*lin_terms_W_minus_list[i]],[0.5*lin_terms_W_minus_list[i].conj().T, -cst_terms[i]]]) for i in range(len(lin_terms_W_minus_list))]
sdr_mat_J_def = [sparse.bmat([[J_def_quad[i], 0.5*lin_terms_J_def[i]],[0.5*lin_terms_J_def[i].conj().T, cst_terms_def[i]]]) for i in range(len(lin_terms_W_minus_list))]

In [5]:
obj_quad = sparse.kron(sparse.eye(N_omega), -sparse.eye(N_omega))
obj_quad.resize((3*(N_omega**2), 3*(N_omega**2)))
obj_quad = sparse.bmat([[obj_quad, sparse.csc_matrix((3*(N_omega**2), 1))],[sparse.csc_matrix((1,3*(N_omega**2))), n]])

In [6]:
constraint_unit = sparse.bmat([[sparse.csc_matrix((3*(N_omega**2), 3*(N_omega**2))), sparse.csc_matrix((3*(N_omega**2), 1))],[sparse.csc_matrix((1, 3*(N_omega**2))), sparse.eye(1)]])

In [7]:
total_constraints = sdr_mat_J_def + sdr_mat_W_plus + sdr_mat_W_minus
total_constraints.append(constraint_unit)
variable = cp.Variable(shape = (3*(N_omega**2) + 1, 3*(N_omega**2) + 1))
constraints = [variable >> 0]
constraints += [cp.trace(sdr_mat_J_def[i]@variable) == 0 for i in range(len(sdr_mat_J_def))]
constraints += [cp.trace(sdr_mat_W_plus[i]@variable) == 0 for i in range(len(sdr_mat_W_plus))]
constraints += [cp.trace(sdr_mat_W_minus[i]@variable) == 0 for i in range(len(sdr_mat_W_minus))]
constraints.append(cp.trace(constraint_unit@variable) == 1)
problem = cp.Problem(cp.Minimize(cp.trace(obj_quad@variable)), constraints)

In [8]:
[(y.conj().T@total_constraints[i]@y).toarray()[0,0] for i in range(len(total_constraints)) if np.abs((y.conj().T@total_constraints[i]@y).toarray()[0,0]) > 10**-9]

[1.0]

In [9]:
problem.solve(solver = "CLARABEL", verbose = True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Feb 07 03:10:05 PM: Your problem has 169 variables, 14 constraints, and 0 parameters.
(CVXPY) Feb 07 03:10:05 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 07 03:10:05 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 07 03:10:05 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 07 03:10:05 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 07 03:10:05 PM: Compiling problem (target solver=CLARABEL)

-inf